In [10]:
# составляем корпус документов

import os

curr_dir = os.getcwd()
dirpath = os.path.join(curr_dir, 'friends-data')
fnames = []
corpus = []

for root, dirs, files in os.walk(dirpath):
    for name in files:
        fnames.append(os.path.join(root, name))


for fpath in fnames:
    with open(fpath, 'r', encoding='utf-8') as f:
        text = ''.join([str(item) for item in f.readlines()])
        corpus.append(text)

#проверка
print(len(corpus))

165


In [13]:
!pip install pymorphy2

  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=c8a0c0eb8db422263cb506c58da6649ab34400618c1f009a36ed222d6b88ef6d
  Stored in directory: c:\users\vfif\appdata\local\pip\cache\wheels\56\ea\58\ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [58]:
import nltk
import pymorphy2
import re

nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
morph = pymorphy2.MorphAnalyzer()

stop_words = stopwords.words('russian')

#предобработка текстов

prep_corpus = []

for i, item in enumerate(corpus):
    filtered_tokens = []
    final_tokens = []
    item = item.lower()
    tokens = word_tokenize(item, language='russian')
    for token in tokens:
        if token not in stop_words and re.search(r'\w',token):
            filtered_tokens.append(token)
    for token in filtered_tokens:
        final_tokens.append(morph.parse(token)[0].normal_form)
    text = ' '.join(final_tokens)
    prep_corpus.append(text)
print(prep_corpus)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vfif\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vfif\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['\ufeffдруг начаться нечего рассказывать просто сотрудник ладный ты свидание собраться значит придурь джой вести прилично горб парик придача погодить знать есть мел знать просто хотеть пережить карлом… ой ладный успокоиться успокоиться это свидание просто двое молодой человек идти поужинать вместе секс такой снова старшеклассник стоить посреди столовая понимать полностью голый мы это сниться значит смотреть вниз видеть телефон вместо именно такой сниться внезапно телефон начинать звонить знать делать смотреть смотреть догадываться ответить оказываться это мать это-то самый странный звонить здороваться ответ хотеться повеситься порядок милый чувствовать вроде кто-то залезть к глотка схватить кишка вытащить рот обмотать вокруг шея печенька кэрол сегодня переехать вещь забрать давать-ка ты кофеёк принести спасибо хватить чистить аура оставить мой аура покой ладный ладный ходить чмо понять надеяться счастливый надеяться надеяться пусть провалиться ад бросить знать лесбиянка ладный ради за

In [19]:
print(len(prep_corpus))

165


In [76]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
 
# инициализируем
vectorizer = CountVectorizer(analyzer='word')

# считаем
X = vectorizer.fit_transform(prep_corpus)
print('X:\n', X.toarray(), '\n')
# получится следующая структура:
#        | слово1 | слово2 | слово3 | слово4
# текст1 |   1    |    1   |   1    |   0
# текст2 |   0    |    1   |   1    |   0
# текст3 |   2    |    1   |   0    |   0
# текст4 |   0    |    0   |   0    |   1

#ft = vectorizer.get_feature_names()
#df = pd.DataFrame(X, columns=ft)

matrix_freq = np.asarray(X.sum(axis=0)).ravel()
final_matrix = np.array([np.array(vectorizer.get_feature_names()), matrix_freq])
print('final_matrix:', final_matrix, '\n')  # результат [['слово1' 'слово2' 'слово3' 'слово4'], ['3' '3' '2' '1']] 


X:
 [[5 0 0 ... 0 0 0]
 [5 0 0 ... 0 0 0]
 [5 0 0 ... 0 0 0]
 ...
 [5 0 0 ... 0 0 0]
 [5 0 0 ... 0 0 0]
 [5 0 0 ... 0 0 0]] 

final_matrix: [['00' '000' '007' ... 'ёрш' 'ёршик' 'ёще']
 ['830' '9' '7' ... '1' '1' '1']] 



In [105]:
counts = pd.DataFrame(X.toarray(),
                      columns=vectorizer.get_feature_names())

#какое слово является самым частотным

counts.T.sort_values(by=0, ascending=False).head(10)

#ответ а) это

,0,1,2,3,4,5,6,7,8,9,...,155,156,157,158,159,160,161,162,163,164
это,58,28,46,29,39,42,40,23,22,27,...,60,43,30,32,27,52,38,30,17,24
знать,30,14,12,9,14,13,10,18,13,12,...,8,13,6,9,10,10,38,11,7,18
просто,20,5,8,3,11,11,9,9,4,6,...,5,6,2,2,3,8,2,3,3,5
ты,20,9,21,7,17,7,12,9,8,14,...,8,14,7,12,8,9,10,12,8,10
пол,19,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,2
то,18,9,9,12,6,8,9,12,5,7,...,6,9,9,4,5,9,5,4,10,4
мочь,14,2,12,10,5,9,3,7,6,10,...,14,8,4,10,6,5,11,5,14,12
хотеть,13,12,12,7,10,12,9,7,14,13,...,8,12,9,8,8,14,8,14,7,10
сказать,11,6,7,10,3,6,12,11,8,18,...,7,10,5,9,12,6,13,3,10,4
свидание,10,0,2,0,2,2,0,2,0,1,...,4,0,7,1,1,0,0,0,1,0


In [106]:
#какое слово является самым редким

counts.T.sort_values(by=0, ascending=True).head(10)

#Ответ b) ёще

,0,1,2,3,4,5,6,7,8,9,...,155,156,157,158,159,160,161,162,163,164
ёще,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
поручать,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
поручение,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
поручить,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
порушить,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
порша,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0
порше,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
поршировать,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
порыться,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
чудовищно,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#какой набор слов есть во все документах коллекции



In [108]:
#кто из главных героев упоминается чаще всего

heroes_freq = []

#Моника
print(counts['моника'] + counts['мон'])

#Рэйчел
print(counts['рэйчел'] + counts['рейч'])

#Чендлер
print(counts['чендлер'] + counts['чен'])

#Фиби
print(counts['фиби'])

#Росс
print(counts['росс'])

#Джоуи
print(counts['джой'] + counts['джоуи'])


0       5
1       2
2       1
3       3
4       6
       ..
160     6
161    18
162     7
163    14
164    10
Length: 165, dtype: int64
0      4
1      2
2      0
3      5
4      3
      ..
160    1
161    0
162    2
163    2
164    2
Length: 165, dtype: int64
0      0
1      0
2      0
3      0
4      0
      ..
160    0
161    0
162    0
163    1
164    0
Length: 165, dtype: int64
0      1
1      0
2      4
3      3
4      0
      ..
160    3
161    6
162    3
163    1
164    3
Name: фиби, Length: 165, dtype: int64
0      3
1      5
2      3
3      4
4      3
      ..
160    6
161    3
162    2
163    8
164    1
Name: росс, Length: 165, dtype: int64
0      6
1      0
2      0
3      3
4      3
      ..
160    0
161    0
162    0
163    0
164    0
Length: 165, dtype: int64


In [59]:
print(morph.parse('джои')[0].normal_form)

джой
